## Load in required libraries and API keys

In [130]:
import json
import pickle
import operator
import requests

with open('config.json') as json_data:
    d = json.load(json_data)
    DRIBBBLE_API_KEY = d['dribbble-key']

Idea of cliches in data
- Kapil might try to scrape out all the users...

Interesting questions about rebounds:
- If you get only one rebound, how often are these self-rebound?
- Difference tags among rebounded and non-rebounded work

Other interesting things
- Reciprocity in comments (if I comment on someone, do they comment back?) --> use adj matrix with all people who have commented on other people's work
- Is a person more likely to give more comments if they get more comments? 

In [131]:
all_shots = []

page_number = 1
while True:    
    print 'Current page: ' + str(page_number)
    shots_per_page_response = requests.get('https://api.dribbble.com/v1/shots', 
                                           headers={'Authorization': 'Bearer ' + DRIBBBLE_API_KEY},
                                           params={'timeframe': 'ever', 'page': page_number, 'per_page': 100})

    if shots_per_page_response.status_code == 200:
        all_shots.extend(shots_per_page_response.json())
        page_number += 1
    else:
        break
        
with open('all_shots.dat', 'wb') as f:
    pickle.dump(all_shots, f)

Current page: 1
Current page: 2
Current page: 3
Current page: 4
Current page: 5
Current page: 6
Current page: 7
Current page: 8
Current page: 9
Current page: 10
Current page: 11
Current page: 12
Current page: 13
Current page: 14
Current page: 15
Current page: 16
Current page: 17
Current page: 18
Current page: 19
Current page: 20
Current page: 21
Current page: 22
Current page: 23
Current page: 24
Current page: 25
Current page: 26
Current page: 27
Current page: 28
Current page: 29
Current page: 30
Current page: 31
Current page: 32
Current page: 33
Current page: 34
Current page: 35
Current page: 36
Current page: 37
Current page: 38
Current page: 39
Current page: 40
Current page: 41
Current page: 42
Current page: 43
Current page: 44
Current page: 45
Current page: 46
Current page: 47
Current page: 48
Current page: 49
Current page: 50
Current page: 51


In [132]:
shot_data = []
for shot in all_shots:
    current_data = {
        'rebounds': shot['rebounds_count'],
        'is_rebound': shot['rebounds_count'] > 0,
        'tags': shot['tags'],
        'tag_count': len(shot['tags']),
        'follower_count': shot['user']['followers_count'],
        'following_count': shot['user']['followings_count'],
        'views': shot['views_count'],
        'likes': shot['likes_count'],
        'comment_count': shot['comments_count'],
        'is_pro': shot['user']['pro']
    }
    
    shot_data.append(current_data)

In [133]:
rebound_tag_counts = {}
non_rebound_tag_counts = {}

for shot in shot_data:
    if shot['is_rebound']:
        for tag in shot['tags']:
            if tag in rebound_tag_counts:
                rebound_tag_counts[tag] += 1
            else:
                rebound_tag_counts[tag] = 1
    else:
        for tag in shot['tags']:
            if tag in non_rebound_tag_counts:
                non_rebound_tag_counts[tag] += 1
            else:
                non_rebound_tag_counts[tag] = 1                

In [134]:
sorted_rebound_tag_counts = sorted(rebound_tag_counts.items(), key=operator.itemgetter(1), reverse=True)
sorted_non_rebound_tag_counts = sorted(non_rebound_tag_counts.items(), key=operator.itemgetter(1), reverse=True)

In [135]:
sorted_rebound_tag_counts

[(u'illustration', 414),
 (u'ui', 382),
 (u'icon', 282),
 (u'app', 214),
 (u'logo', 209),
 (u'ux', 207),
 (u'ios', 191),
 (u'animation', 184),
 (u'vector', 175),
 (u'design', 163),
 (u'flat', 161),
 (u'icons', 152),
 (u'iphone', 139),
 (u'gif', 134),
 (u'web', 116),
 (u'clean', 97),
 (u'interface', 96),
 (u'mobile', 85),
 (u'website', 83),
 (u'typography', 78),
 (u'minimal', 75),
 (u'branding', 73),
 (u'sketch', 67),
 (u'dashboard', 65),
 (u'identity', 65),
 (u'mark', 63),
 (u'3d', 62),
 (u'character', 61),
 (u'space', 57),
 (u'simple', 56),
 (u'type', 55),
 (u'free', 55),
 (u'navigation', 55),
 (u'texture', 55),
 (u'apple', 52),
 (u'blue', 50),
 (u'freebie', 48),
 (u'psd', 47),
 (u'motion', 46),
 (u'concept', 45),
 (u'button', 45),
 (u'card', 44),
 (u'layout', 38),
 (u'badge', 38),
 (u'map', 38),
 (u'music', 36),
 (u'menu', 36),
 (u'graph', 36),
 (u'social', 36),
 (u'after effects', 35),
 (u'photoshop', 35),
 (u'sun', 34),
 (u'ipad', 34),
 (u'car', 33),
 (u'white', 33),
 (u'material',

In [136]:
sorted_non_rebound_tag_counts

[(u'illustration', 786),
 (u'ui', 649),
 (u'animation', 530),
 (u'logo', 402),
 (u'app', 397),
 (u'icon', 392),
 (u'design', 364),
 (u'ux', 354),
 (u'gif', 323),
 (u'ios', 301),
 (u'flat', 279),
 (u'web', 278),
 (u'typography', 232),
 (u'iphone', 218),
 (u'branding', 217),
 (u'icons', 202),
 (u'website', 197),
 (u'vector', 183),
 (u'interface', 177),
 (u'motion', 174),
 (u'sketch', 170),
 (u'mobile', 148),
 (u'character', 147),
 (u'clean', 140),
 (u'identity', 139),
 (u'minimal', 129),
 (u'3d', 119),
 (u'dashboard', 116),
 (u'texture', 108),
 (u'type', 105),
 (u'space', 100),
 (u'mark', 95),
 (u'paper', 93),
 (u'vintage', 91),
 (u'layout', 88),
 (u'graph', 87),
 (u'after effects', 86),
 (u'focus lab', 85),
 (u'logotype', 81),
 (u'ipad', 80),
 (u'badge', 79),
 (u'blue', 79),
 (u'game', 78),
 (u'simple', 76),
 (u'wood', 74),
 (u'menu', 73),
 (u'lettering', 73),
 (u'landing', 73),
 (u'print', 71),
 (u'interaction', 70),
 (u'navigation', 70),
 (u'retro', 69),
 (u'web design', 67),
 (u'land

In [137]:
shots_with_rebounds = 0
for shot in all_shots:
    if shot['rebounds_count'] != 0:
        shots_with_rebounds += 1

shots_with_rebounds

1684